In [8]:
#dependencies
from pint import UnitRegistry
import numpy as np

#unit setup
ureg = UnitRegistry()
m = ureg.meter
g = ureg.gram
s = ureg.second
rpm = ureg.rpm
N = ureg.newton
rho_air = ureg.atm
#variable array setup
speed = [1500, 2000, 2500, 3000, 3500, 4000]
torque = [1.8, 1.8, 1.8, 1.8, 1.8, 1.8]
power_shaft = [None, None, None, None, None, None] # not provided in the given table
power_input = [5.55E-05, 5.64E-05, 6.83E-05, 8.26E-05, 0.000104, 0.000118]
efficiency = [None, None, None, None, None, None] # not provided in the given table
MEP = [247, 247, 247, 291, 330, 350]
bsfc = [None, None, None, None, None, None] # not provided in the given table
exhaust_heat = [None, None, None, None, None, None] # not provided in the given table
fins_heat = [None, None, None, None, None, None] # not provided in the given table
air_mass_flow = [None, None, None, None, None, None] # not provided in the given table
fuel_mass_flow = [None, None, None, None, None, None] # not provided in the given table
deltaT = [None, None, None, None, None, None] # not provided in the given table

In [9]:

D = 10e-4 * m # displacement of the engine
N = speed * rpm #engine speed in rotations per minute
mdot_air = (rho_air * D * N) / 120 # 120 is product of 2 and 60, a constant for four stroke engines


TypeError: can't multiply sequence by non-int of type 'Quantity'